In [1]:
import pyodbc
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')

from warnings import\
    filterwarnings as FilterOutWarnings
FilterOutWarnings("ignore")

from math import sqrt
from statsmodels.formula import\
    api as sm
from scipy.stats import\
    ttest_ind as T_test,\
    f_oneway as F_test
from sklearn.metrics import\
    mean_squared_error as MSE,\
    mean_absolute_error as MAE,\
    r2_score as R2

In [2]:
def getDataFrameFromSqlServer(sql_select):
    if (len(sql_select) == 0 or sql_select == ""):
        return pd.DataFrame([])
    
    connStr = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=INBOOK_X1\MSSQLSERVER2;DATABASE=PrayerTimesDW;UID=sa;PWD=abcdE!2345;"
    conn = pyodbc.connect(connStr)
    try:
        return pd.read_sql(sql=sql_select, con=conn)
    except Exception as ex:
        print(ex)
        return pd.DataFrame([])

In [3]:
df_train = getDataFrameFromSqlServer\
    ("""
        SELECT time_date, time_fajr, time_zuhr, time_asr, time_maghrib, time_isha 
        FROM t_time_prayer_source_1 WITH(NOLOCK)
        WHERE time_date <= '2023-12-31'
        ORDER BY time_date ASC;
    """)

df_train

,time_date,time_fajr,time_zuhr,time_asr,time_maghrib,time_isha
0,2018-01-01,05:58:00,13:19:00,16:42:00,19:16:00,20:30:00
1,2018-01-02,05:58:00,13:20:00,16:42:00,19:16:00,20:31:00
2,2018-01-03,05:59:00,13:20:00,16:43:00,19:17:00,20:31:00
3,2018-01-04,05:59:00,13:21:00,16:43:00,19:17:00,20:32:00
4,2018-01-05,06:00:00,13:21:00,16:44:00,19:18:00,20:32:00
...,...,...,...,...,...,...
2186,2023-12-27,06:04:00,13:17:00,16:39:00,19:14:00,20:29:00
2187,2023-12-28,06:04:00,13:17:00,16:40:00,19:14:00,20:29:00
2188,2023-12-29,06:05:00,13:18:00,16:40:00,19:15:00,20:30:00
2189,2023-12-30,06:05:00,13:18:00,16:41:00,19:15:00,20:30:00


In [4]:
df_test = getDataFrameFromSqlServer\
    ("""SELECT time_date, time_fajr, time_zuhr, time_asr, time_maghrib, time_isha 
        FROM t_time_prayer_source_1 WITH(NOLOCK)
        WHERE time_date > '2023-12-31'
        ORDER BY time_date ASC;
    """)

df_test

,time_date,time_fajr,time_zuhr,time_asr,time_maghrib,time_isha
0,2024-01-01,06:06:00,13:19:00,16:42:00,19:16:00,20:31:00
1,2024-01-02,06:07:00,13:20:00,16:42:00,19:17:00,20:31:00
2,2024-01-03,06:07:00,13:20:00,16:43:00,19:17:00,20:32:00
3,2024-01-04,06:08:00,13:21:00,16:43:00,19:18:00,20:32:00
4,2024-01-05,06:08:00,13:21:00,16:44:00,19:18:00,20:33:00
...,...,...,...,...,...,...
361,2024-12-27,06:04:00,13:17:00,16:40:00,19:14:00,20:29:00
362,2024-12-28,06:05:00,13:18:00,16:40:00,19:15:00,20:30:00
363,2024-12-29,06:05:00,13:18:00,16:41:00,19:15:00,20:30:00
364,2024-12-30,06:06:00,13:19:00,16:41:00,19:16:00,20:30:00


In [5]:
total_data = len(df_train) + len(df_test)
train_pct = len(df_train) / total_data
test_pct = len(df_test) / total_data
print(f"train data to total (out of 1.00): {train_pct:.2f}")
print(f"test data to total (out of 1.00): {test_pct:.2f}")

train data to total (out of 1.00): 0.86
test data to total (out of 1.00): 0.14
